In [2]:
import numpy as np
import pandas as pd
import re, nltk, spacy, gensim

# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt

%matplotlib inline

/Users/yanglingxi/opt/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
cd /Users/yanglingxi/Desktop/deloitte

/Users/yanglingxi/Desktop/deloitte


/Users/yanglingxi/opt/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
# Import Dataset
df= pd.read_csv('USAID_Anticorruption_Projects_Database.csv')
df.drop(['proj_res','rfp_rftop','final_res','mind_eval','final_eval','audit','interim_rep','addl_docs','addl_info'],axis=1,inplace=True)

/Users/yanglingxi/opt/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
# Convert to list
data = df.proj_desc.values.tolist()

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['The five components of this program seek to take what already exists and '
 'improve upon it, implement it and find ways to ensure that the progress '
 'achieved is sustainable over the long term. The project will help the '
 'government of the Islamic Republic of Afghanistan strengthen its High Office '
 'of Oversight (HOO) - making it a strong, effective institution that is able '
 'to lead, monitor, coordinate and report on efforts to combat corruption '
 'across the country. The project will work and implement a program of '
 'institutional development and sustainability. It will also support the '
 'office in carrying out its priority responsibilities in asset registration '
 'and verification, complaints management and case tracking, and coordination '
 'and monitoring of anticorruption performance across other government '
 'agencies.']


/Users/yanglingxi/opt/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
#tokenize and clean up using gensim 

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])


/Users/yanglingxi/opt/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[['the', 'five', 'components', 'of', 'this', 'program', 'seek', 'to', 'take', 'what', 'already', 'exists', 'and', 'improve', 'upon', 'it', 'implement', 'it', 'and', 'find', 'ways', 'to', 'ensure', 'that', 'the', 'progress', 'achieved', 'is', 'sustainable', 'over', 'the', 'long', 'term', 'the', 'project', 'will', 'help', 'the', 'government', 'of', 'the', 'islamic', 'republic', 'of', 'afghanistan', 'strengthen', 'its', 'high', 'office', 'of', 'oversight', 'hoo', 'making', 'it', 'strong', 'effective', 'institution', 'that', 'is', 'able', 'to', 'lead', 'monitor', 'coordinate', 'and', 'report', 'on', 'efforts', 'to', 'combat', 'corruption', 'across', 'the', 'country', 'the', 'project', 'will', 'work', 'and', 'implement', 'program', 'of', 'institutional', 'development', 'and', 'sustainability', 'it', 'will', 'also', 'support', 'the', 'office', 'in', 'carrying', 'out', 'its', 'priority', 'in', 'asset', 'registration', 'and', 'verification', 'complaints', 'management', 'and', 'case', 'tracking

In [7]:
#lemmatization

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# Run in terminal: python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:2])

/Users/yanglingxi/opt/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['component program seek take already exist improve implement find way ensure progress achieve sustainable long term project will help government strengthen high office oversight hoo make strong effective institution able lead monitor coordinate report effort combat corruption country project will work implement program institutional development sustainability will also support office carry priority asset registration verification complaint management case tracking coordination monitoring anticorruption performance other government agency', 'overall goal program support rule law strategy specifically pillar improve justice sector reduce corruption pillar build leadership justice sector civil society support pillar contractor attempt develop capacity judiciary law school pillar contractor work raise public awareness encourage citizen resolve dispute formal justice sector program divide component capacity building judiciary capacity building court management system capacity building facu

In [8]:
## TFIDF


vectorizer = TfidfVectorizer(analyzer='word',       
                             min_df=10,                        # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             ngram_range=(1,3)             # max number of uniq words
                            )

data_vectorized = vectorizer.fit_transform(data_lemmatized)

/Users/yanglingxi/opt/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
#Checking the sparcity

# Materialize the sparse data
data_dense = data_vectorized.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

Sparsicity:  9.346462864981383 %


/Users/yanglingxi/opt/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
## BASE MODEL



# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=20,               # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

/Users/yanglingxi/opt/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


LatentDirichletAllocation(learning_method='online', n_components=20, n_jobs=-1,
                          random_state=100)


In [11]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(data_vectorized))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))

# See model parameters
pprint(lda_model.get_params())

Log Likelihood:  -11555.739701306189


/Users/yanglingxi/opt/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Perplexity:  1993.3942208330284
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 20,
 'n_jobs': -1,
 'perp_tol': 0.1,
 'random_state': 100,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


In [12]:
## HYPERPARAMETER TUNING

# Define Search Param
search_params = {'n_components': np.arange(5,30).tolist(), 'learning_decay': [.3,.5, .7, .9]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
model.fit(data_vectorized)

/Users/yanglingxi/opt/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


GridSearchCV(estimator=LatentDirichletAllocation(),
             param_grid={'learning_decay': [0.3, 0.5, 0.7, 0.9],
                         'n_components': [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,
                                          16, 17, 18, 19, 20, 21, 22, 23, 24,
                                          25, 26, 27, 28, 29]})

In [13]:
## BEST TOPIC MODEL


# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))



Best Model's Params:  {'learning_decay': 0.3, 'n_components': 5}
Best Log Likelihood Score:  -2451.526240918424
Model Perplexity:  775.3861757911554


/Users/yanglingxi/opt/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [14]:
### DOMINANT TOPIC IN EACH DOCUMENT


# Create Document - Topic Matrix
lda_output = best_lda_model.transform(data_vectorized)

# column names
topicnames = ["Topic" + str(i) for i in range(best_lda_model.n_components)]

# index names
docnames = ["Doc" + str(i) for i in range(len(data))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics




/Users/yanglingxi/opt/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Topic0,Topic1,Topic2,Topic3,Topic4,dominant_topic
Doc0,0.030000,0.030000,0.030000,0.030000,0.890000,4
Doc1,0.030000,0.030000,0.030000,0.410000,0.490000,4
Doc2,0.050000,0.050000,0.050000,0.810000,0.050000,3
Doc3,0.040000,0.040000,0.120000,0.040000,0.740000,4
Doc4,0.040000,0.040000,0.040000,0.460000,0.430000,3
Doc5,0.030000,0.030000,0.030000,0.030000,0.870000,4
Doc6,0.030000,0.030000,0.030000,0.030000,0.880000,4
Doc7,0.040000,0.040000,0.080000,0.040000,0.800000,4
Doc8,0.040000,0.040000,0.040000,0.040000,0.860000,4
Doc9,0.030000,0.030000,0.030000,0.030000,0.880000,4


In [15]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(best_lda_model, data_vectorized, vectorizer, mds='tsne')
panel

/Users/yanglingxi/opt/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
4      150.222153  -72.269554       1        1  62.779574
3       51.163364 -159.968018       2        1  25.433260
2      -73.144554   21.523815       3        1   4.403371
1      -66.410133 -110.606438       4        1   3.974417
0       54.417324   12.443444       5        1   3.409377, topic_info=                     Term      Freq     Total Category  logprob  loglift
134                health  5.000000  5.000000  Default  30.0000  30.0000
25             assessment  5.000000  5.000000  Default  29.0000  29.0000
305                   tax  3.000000  3.000000  Default  28.0000  28.0000
231           procurement  5.000000  5.000000  Default  27.0000  27.0000
169               justice  9.000000  9.000000  Default  26.0000  26.0000
..                    ...       ...       ...      ...      ...      ...
14                 afghan  0.160025  3.365244   Topic5  -5.7809   0.3327
173                   law  0.160015  8.780971   Topic5  -5.7810  -0.6264
199               network  0.160013  3.383701   Topic5  -5.7810   0.3272
91   economic development  0.160012  2.851220   Topic5  -5.7810   0.4984
114            facilitate  0.160010  2.949791   Topic5  -5.7810   0.4644

[282 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
1         1  0.361293          access
1         2  0.541939          access
2         1  0.774632  accountability
2         2  0.129105  accountability
4         1  0.516122         achieve
...     ...       ...             ...
320       1  0.905174             use
321       1  0.917227           woman
322       1  0.841325            work
322       2  0.120189            work
323       1  0.716867            year

[272 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 4, 3, 2, 1])

In [16]:
### TOPIC KEYWORDS ###

# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(best_lda_model.components_)

# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topicnames

# View
df_topic_keywords.head()

/Users/yanglingxi/opt/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,ability,access,accountability,accountable,achieve,action,activity,actor,address,administration,...,training,transition,transparency,transparency accountability,transparent,unit,use,woman,work,year
Topic0,0.200170,0.200171,0.200148,0.200159,0.200171,0.200173,0.200149,0.200277,0.200176,0.200174,...,0.200149,0.200145,0.200162,0.200163,0.200153,0.200190,0.202068,0.200126,0.200162,0.204195
Topic1,0.200148,0.200704,0.201139,0.200136,0.204247,0.200572,0.200464,0.200237,0.200149,0.285080,...,0.200375,0.200123,0.206170,0.202619,0.200243,0.200165,0.207878,0.200177,0.200560,0.200957
Topic2,0.200296,0.201400,0.200366,0.200119,0.200534,0.201948,0.200332,0.620140,0.202095,0.200130,...,0.200218,0.200107,0.200208,0.200362,0.200154,0.200794,0.201073,0.200860,0.200703,0.210839
Topic3,1.672589,3.253618,1.692621,1.081156,1.251673,0.537532,2.018454,0.373058,1.506041,2.373422,...,2.446377,0.203318,2.394370,0.700667,1.148181,0.655546,0.370514,0.201337,1.556057,0.202255
Topic4,1.201845,2.914411,7.024572,3.404151,2.845492,2.016174,4.937615,2.578471,3.888574,3.175796,...,5.837550,2.875085,8.340261,4.208488,4.798523,1.692908,4.324336,4.425645,7.837008,2.545687


In [17]:
### TOP 15 KEYWORDS FOR EACH TOPIC ###

# Show top n keywords for each topic
def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=vectorizer, lda_model=best_lda_model, n_words=15)        

# Topic - Keywords Dataframe
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

/Users/yanglingxi/opt/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,non,governmental,anticorruption,year,report,use,corruption,focus,organization,support implementation,enforcement,program focus,program,government institution,streamline
Topic 1,tax,public procurement,procurement,reduce corruption,reduce,overall,enforcement,great,program,improve public,corruption,opportunity,administration,approach,efficiency
Topic 2,health,assessment,identify,political,right,corruption,project design,stakeholder,base,design,actor,country,opportunity,strategy,conduct
Topic 3,justice,judicial,law,legal,court,rule,reform,improve,sector,trade,rule law,project,support,strengthen,increase
Topic 4,local,government,corruption,project,program,governance,capacity,public,support,improve,service,strengthen,civil,society,aim


In [22]:
cd /Users/yanglingxi/Desktop

/Users/yanglingxi/Desktop


/Users/yanglingxi/opt/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [23]:
df_document_topic.to_csv('df_document_topic.csv')

/Users/yanglingxi/opt/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
